In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import GlobalAveragePooling2D
from keras import Sequential
import efficientnet.keras as efn
import numpy as np
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import to_categorical
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, f1_score, precision_score, recall_score, precision_recall_curve
import seaborn as sns
import datetime


: 

In [ ]:
import os
import pandas as pd

def count_images_per_class(folder_path):
    class_counts = {}
    for class_name in sorted(os.listdir(folder_path)):
        class_dir = os.path.join(folder_path, class_name)
        if os.path.isdir(class_dir):
            count = len([
                f for f in os.listdir(class_dir)
                if os.path.isfile(os.path.join(class_dir, f))
            ])
            class_counts[class_name] = count
    return class_counts

# Count images in each split
train_counts = count_images_per_class('split_dataset/train')
val_counts = count_images_per_class('split_dataset/val')
test_counts = count_images_per_class('split_dataset/test')

# Create summary DataFrame
summary_df = pd.DataFrame({
    'train': train_counts,
    'val': val_counts,
    'test': test_counts
}).fillna(0).astype(int).T

summary_df = summary_df.T  # Transpose to get classes as rows
summary_df



Load Data

In [ ]:
IMAGE_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1. / 255)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'split_dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

val_generator = val_test_datagen.flow_from_directory(
    'split_dataset/val',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    'split_dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

In [ ]:
# Create base DenseNet model
base_model = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'  # Global Average Pooling for output (1024,)
)
base_model.trainable = False  # Freeze base model initially

model = Model(inputs=base_model.input, outputs=base_model.output)


In [ ]:
"""nb_eval_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)
predict_size = int(math.ceil(nb_eval_samples / BATCH_SIZE))

bottleneck_features = model.predict(train_generator, verbose=1)
print(bottleneck_features.shape)

nb_eval_samples_val = len(val_generator.filenames)
num_classes_val = len(val_generator.class_indices)
predict_size_val = int(math.ceil(nb_eval_samples_val / BATCH_SIZE))

bottleneck_features_val = model.predict(val_generator, verbose=1)
print(bottleneck_features_val.shape)

nb_eval_samples_test = len(test_generator.filenames)
num_classes_test = len(test_generator.class_indices)
predict_size_test = int(math.ceil(nb_eval_samples_test / BATCH_SIZE))

bottleneck_features_test = model.predict(test_generator, verbose=1)
print(bottleneck_features_test.shape)"""

In [ ]:
"""if np.max(train_generator.classes) == 5:
    y = np.array(train_generator.classes) - 1
else:
    y = np.array(train_generator.classes)

np.save('split_dataset/5153_classes_dn121.npy', y)


print(bottleneck_features.shape)
np.save('split_dataset/5153_features_dn121.npy', bottleneck_features)

if np.max(val_generator.classes) == 5:
    y = np.array(val_generator.classes) - 1
else:
    y = np.array(val_generator.classes)

np.save('split_dataset/5153_classes_dn121_val.npy', y)


print(bottleneck_features_val.shape)
np.save('split_dataset/5153_features_dn121_val.npy', bottleneck_features_val)

if np.max(test_generator.classes) == 5:
    y = np.array(test_generator.classes) - 1
else:
    y = np.array(test_generator.classes)

np.save('split_dataset/5153_classes_dn121_test.npy', y)


print(bottleneck_features_test.shape)
np.save('split_dataset/5153_features_dn121_test.npy', bottleneck_features_test)"""


In [ ]:
X_train = np.load('split_dataset/5153_features_dn121.npy')
y_train = np.load('split_dataset/5153_classes_dn121.npy')

print("Train Features shape:", X_train.shape)  # e.g., (num_samples, 7, 7, 1280)
print("Train Labels shape:", y_train.shape)

X_val = np.load('split_dataset/5153_features_dn121_val.npy')
y_val = np.load('split_dataset/5153_classes_dn121_val.npy')

print("Val Features shape:", X_val.shape)  # e.g., (num_samples, 7, 7, 1280)
print("Val Labels shape:", y_val.shape)

X_test = np.load('split_dataset/5153_features_dn121_test.npy')
y_test = np.load('split_dataset/5153_classes_dn121_test.npy')

print("Test Features shape:", X_test.shape)  # e.g., (num_samples, 7, 7, 1280)
print("Test Labels shape:", y_test.shape)

In [ ]:
# One-hot encode labels
from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(y_train))

# Assuming y_train and y_val are 1D integer class labels (0 to 4)
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)

In [ ]:
# Make sure your feature extractor outputs the correct dimensions
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
print(f"Feature extractor output shape: {feature_extractor.output_shape}")  # Should print (None, 1024)

Build & Train Model

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.activations import gelu

# Example: Different alphas for classes (e.g., [0.25, 0.75, 1.0, ...])
alpha_per_class = tf.constant([0.25, 0.7, 1.0, 0.8, 0.9], dtype=tf.float32)

def focal_loss_with_smoothing_and_dynamic_alpha(alpha_per_class, gamma=2.0, smoothing=0.1):
    alpha_per_class = tf.constant(alpha_per_class, dtype=tf.float32)
    
    def loss_fn(y_true, y_pred):
        num_classes = tf.cast(tf.shape(y_true)[-1], dtype=tf.float32)
        y_true = y_true * (1.0 - smoothing) + smoothing / num_classes

        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        cross_entropy = -y_true * tf.math.log(y_pred)
        focal_weight = tf.pow(1. - y_pred, gamma)
        
        # Apply alpha weight per sample based on true class
        alpha_weight = tf.reduce_sum(alpha_per_class * y_true, axis=-1, keepdims=True)
        loss = focal_weight * cross_entropy * alpha_weight

        return tf.reduce_sum(loss, axis=-1)
    
    return loss_fn

def confidence_penalty_loss(base_loss_fn, beta=0.01):
    def loss_fn(y_true, y_pred):
        base_loss = base_loss_fn(y_true, y_pred)
        
        # Entropy: -sum(p * log(p))
        entropy = -tf.reduce_sum(y_pred * tf.math.log(y_pred + K.epsilon()), axis=-1)
        
        return base_loss + beta * entropy
    return loss_fn

# Define class-wise alpha weights (e.g. based on inverse class frequency)
alpha_per_class = [1.0, 2.0, 1.5, 0.7, 1.2]  # Example for 5 classes — tune as needed

# Create focal loss with smoothing and dynamic alpha
base_loss = focal_loss_with_smoothing_and_dynamic_alpha(alpha_per_class, gamma=2.0, smoothing=0.1)

# Wrap it with confidence penalty
final_loss = confidence_penalty_loss(base_loss, beta=0.01)

# COMPUTE CLASS WEIGHTS
y_train_int = np.argmax(y_train_cat, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_int), y=y_train_int)
class_weights = class_weights / np.max(class_weights) * 3.0
class_weight_dict = dict(enumerate(class_weights))

# Increase weight for class 1 (moderate)
class_weight_dict[1] = class_weight_dict[1] * 1.6  # Boost moderate class
class_weight_dict[3] = class_weight_dict[3] * 1.4  # Higher boost for proliferative class
class_weight_dict[4] = class_weight_dict[4] * 1.2  # Higher boost for severe class

print("Modified class weights:", class_weight_dict)

class KappaHistory(tf.keras.callbacks.Callback):
    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.kappa_train = []
        self.kappa_val = []

    def on_epoch_end(self, epoch, logs=None):
        # Predict class labels for train and val sets
        y_train_pred = np.argmax(self.model.predict(self.X_train, verbose=0), axis=1)
        y_val_pred = np.argmax(self.model.predict(self.X_val, verbose=0), axis=1)
        
        # For the labels, if they are one-hot encoded, you can use argmax to get the class index
        y_train_true = np.argmax(self.y_train, axis=1) if len(self.y_train.shape) > 1 else self.y_train
        y_val_true = np.argmax(self.y_val, axis=1) if len(self.y_val.shape) > 1 else self.y_val

        # Calculate Kappa scores
        train_kappa = cohen_kappa_score(y_train_true, y_train_pred)
        val_kappa = cohen_kappa_score(y_val_true, y_val_pred)

        # Append kappa values
        self.kappa_train.append(train_kappa)
        self.kappa_val.append(val_kappa)

        # Print the Kappa score for the current epoch
        print(f"\nEpoch {epoch + 1} - Train Kappa: {train_kappa:.4f}, Val Kappa: {val_kappa:.4f}")

        # Optionally add kappa to logs for later visualization or saving
        if logs is not None:
            logs['train_kappa'] = train_kappa
            logs['val_kappa'] = val_kappa

# Instantiate the KappaHistory callback
kappa_callback = KappaHistory(X_train, y_train, X_val, y_val)

# Define model using Sequential API
feature_dim = 1024  # DenseNet121 with global average pooling outputs 1024 features
model = models.Sequential([
    layers.InputLayer(input_shape=(feature_dim,)),
    # Deeper network
    #layers.GaussianNoise(0.1),
    layers.Dense(256, activation=tf.nn.swish, kernel_regularizer=regularizers.l2(1e-2)),
    layers.BatchNormalization(momentum=0.95),
    layers.Dropout(0.4),
    layers.Dense(128, activation=tf.nn.swish, kernel_regularizer=regularizers.l2(1e-2)),
    layers.BatchNormalization(momentum=0.95),
    layers.Dropout(0.3),
    layers.Dense(64, activation=tf.nn.swish, kernel_regularizer=regularizers.l2(1e-2)),
    layers.BatchNormalization(momentum=0.95),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

"""# Use Adam optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    weight_decay=1e-4  # Some versions support weight_decay directly
)"""

optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=1e-3,
    rho=0.9,              # Decay factor for moving average of squared gradients
    momentum=0.0,         # Optional momentum term
    epsilon=1e-7,         # Smoothing term to avoid division by zero
    centered=False        # If True, gradients are normalized by estimated variance
)

model.compile(
    optimizer=optimizer,
    loss=final_loss,
    metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc', multi_label=True, num_labels=num_classes)
    ]
)

# Callbacks
output_dir = 'split_dataset/output'
os.makedirs(output_dir, exist_ok=True)

checkpoint = ModelCheckpoint(
    os.path.join(output_dir, 'best_model.h5'),
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Cosine annealing learning rate schedule
import math
def cosine_annealing(epoch, total_epochs=50, initial_lr=3e-4, min_lr=1e-5):
    return min_lr + (initial_lr - min_lr) * (1 + math.cos(math.pi * epoch / total_epochs)) / 2

lr_scheduler = LearningRateScheduler(
    lambda epoch: cosine_annealing(epoch, total_epochs=30, initial_lr=1e-3, min_lr=1e-5)
)

# Train model
history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[checkpoint, early_stopping, lr_scheduler, kappa_callback],
    class_weight=class_weight_dict  # Apply class weights
)

# Plot training history
plt.figure(figsize=(15, 4))

# Accuracy
plt.subplot(1, 4, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')

# Loss
plt.subplot(1, 4, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# AUC
plt.subplot(1, 4, 3)
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend(['Train', 'Validation'], loc='lower right')

# Kappa
plt.subplot(1, 4, 4)
plt.plot(kappa_callback.kappa_train)
plt.plot(kappa_callback.kappa_val)
plt.title("Model Kappa")
plt.xlabel("Epoch")
plt.ylabel("Kappa")
plt.legend(["Train", "Validation"], loc='lower right')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'training_history.png'))
plt.show()

In [ ]:
# After initial head training is complete:

from tensorflow.keras.callbacks import Callback
class KappaHistory(tf.keras.callbacks.Callback):
    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.kappa_train = []
        self.kappa_val = []

    def on_epoch_end(self, epoch, logs=None):
        # Predict class labels for train and val sets
        y_train_pred = np.argmax(self.model.predict(self.X_train, verbose=0), axis=1)
        y_val_pred = np.argmax(self.model.predict(self.X_val, verbose=0), axis=1)
        
        # For the labels, if they are one-hot encoded, you can use argmax to get the class index
        y_train_true = np.argmax(self.y_train, axis=1) if len(self.y_train.shape) > 1 else self.y_train
        y_val_true = np.argmax(self.y_val, axis=1) if len(self.y_val.shape) > 1 else self.y_val

        # Calculate Kappa scores
        train_kappa = cohen_kappa_score(y_train_true, y_train_pred)
        val_kappa = cohen_kappa_score(y_val_true, y_val_pred)

        # Append kappa values
        self.kappa_train.append(train_kappa)
        self.kappa_val.append(val_kappa)

        # Print the Kappa score for the current epoch
        print(f"\nEpoch {epoch + 1} - Train Kappa: {train_kappa:.4f}, Val Kappa: {val_kappa:.4f}")

        # Optionally add kappa to logs for later visualization or saving
        if logs is not None:
            logs['train_kappa'] = train_kappa
            logs['val_kappa'] = val_kappa

# Instantiate the KappaHistory callback
kappa_callback = KappaHistory(X_train, y_train, X_val, y_val)

# Set up ImageDataGenerator for DenseNet121 input
train_datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1. / 255,
    preprocessing_function=tf.keras.applications.densenet.preprocess_input  # Preprocess for DenseNet121
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.densenet.preprocess_input  # Preprocess for DenseNet121
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    'split_dataset/train',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    'split_dataset/val',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Instantiate the KappaHistory callback
kappa_callback = KappaHistory(val_generator)

# Create integrated model with Sequential API
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze all but the last 30 layers
    layer.trainable = False

# Define the integrated model with Sequential API
integrated_model = models.Sequential()
integrated_model.add(base_model)
integrated_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(5e-5)))
integrated_model.add(layers.BatchNormalization(momentum=0.95))
integrated_model.add(layers.Dropout(0.3))
integrated_model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
integrated_model.add(layers.BatchNormalization(momentum=0.95))
integrated_model.add(layers.Dropout(0.2))
integrated_model.add(layers.Dense(num_classes, activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(
    learning_rate=1e-5,     # Lower learning rate for fine-tuning
    rho=0.9,                # Decay factor for moving average of squared gradients
    momentum=0.0,           # Optional momentum term
    epsilon=1e-7,           # Smoothing term to avoid division by zero
    centered=True           # Use centered gradients for stability
)

# Compile with lower learning rate for fine-tuning
integrated_model.compile(
    optimizer=optimizer,
    loss=focal_loss_with_smoothing_and_dynamic_alpha(alpha_per_class, gamma=2.0, smoothing=0.1),
    metrics=['accuracy', 
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc', multi_label=True, num_labels=num_classes)
    ]
)

# Fine-tune
fine_tune_history = integrated_model.fit(
    train_generator,  # Use image data, not features
    validation_data=val_generator,
    epochs=12,
    batch_size=16,
    callbacks=[
        ModelCheckpoint('fine_tuned_model.h5', monitor='val_loss', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        LearningRateScheduler(lambda epoch: 1e-5 * (0.9 ** epoch)),
        kappa_callback
    ],
    class_weight=class_weight_dict
)

In [ ]:
# Plot fine-tuning history
plt.figure(figsize=(15, 4))

# Accuracy
plt.subplot(1, 4, 1)
plt.plot(fine_tune_history.history['accuracy'])
plt.plot(fine_tune_history.history['val_accuracy'])
plt.title('Fine-tuning Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')

# Loss
plt.subplot(1, 4, 2)
plt.plot(fine_tune_history.history['loss'])
plt.plot(fine_tune_history.history['val_loss'])
plt.title('Fine-tuning Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

# AUC
plt.subplot(1, 4, 3)
plt.plot(fine_tune_history.history['auc'])
plt.plot(fine_tune_history.history['val_auc'])
plt.title('Fine-tuning AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend(['Train', 'Validation'], loc='lower right')

# Kappa
plt.subplot(1, 4, 4)
#plt.plot(kappa_callback.kappa_train)
plt.plot(kappa_callback.kappa_val)
plt.title('Fine-tuning Kappa')
plt.xlabel("Epoch")
plt.ylabel("Kappa")
plt.legend(['Train', 'Validation'], loc='lower right')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'fine_tuning_history.png'))
plt.show()

Evaluate Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import label_binarize

# Ensure output dir exists
os.makedirs(output_dir, exist_ok=True)

# Predict probabilities on validation set
print("Running model inference on validation set...")
start_time = time.time()
y_pred_probs = model.predict(X_val, verbose=1)
end_time = time.time()

# Predicted class labels
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Inference time
total_samples = len(X_val)
total_time_ms = (end_time - start_time) * 1000
avg_inference_time = total_time_ms / total_samples

# Compute Cohen's Kappa
kappa_score = cohen_kappa_score(y_val, y_pred_classes)
print(f"\nCohen's Kappa Score: {kappa_score:.4f}")

# Classification report
print("\nClassification Report:")
report_str = classification_report(y_val, y_pred_classes, digits=4)
print(report_str)

with open(os.path.join(output_dir, 'classification_report.txt'), 'w') as f:
    f.write(report_str)

# Confusion matrix
cm = confusion_matrix(y_val, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
plt.show()

# Binarize true labels for ROC curve (needed for multi-class ROC)
y_val_bin = label_binarize(y_val, classes=list(range(num_classes)))

# ROC curve & AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Macro-average ROC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(num_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= num_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
plt.plot(fpr["macro"], tpr["macro"],
         label=f"Macro-average ROC curve (area = {roc_auc['macro']:.2f})",
         color='navy', linestyle='--', linewidth=2)

colors = plt.cm.get_cmap('tab10', num_classes)
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], color=colors(i),
             lw=2, label=f"Class {i} (area = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Multi-class Classification")
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'roc_curves_multiclass.png'))
plt.show()

# Save summary metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics_summary = {
    "accuracy": accuracy_score(y_val, y_pred_classes),
    "macro_precision": precision_score(y_val, y_pred_classes, average='macro'),
    "macro_recall": recall_score(y_val, y_pred_classes, average='macro'),
    "macro_f1": f1_score(y_val, y_pred_classes, average='macro'),
    "macro_roc_auc": roc_auc["macro"],
    "cohen_kappa": kappa_score,  # <-- added here
    "avg_inference_time_ms": avg_inference_time
}

metrics_df = pd.DataFrame(metrics_summary.items(), columns=["Metric", "Value"])
metrics_df.to_csv(os.path.join(output_dir, 'evaluation_metrics.csv'), index=False)

print("\nEvaluation Summary:")
print(metrics_df)

Generate grad_CAM

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np

filepaths = val_generator.filepaths

def load_images_from_paths(paths, target_size=(224, 224)):
    raw_imgs = []
    preprocessed_imgs = []
    for p in paths:
        img = image.load_img(p, target_size=target_size)
        arr = image.img_to_array(img)
        raw_imgs.append(arr / 255.0)
        preprocessed_imgs.append(preprocess_input(arr))
    return np.array(preprocessed_imgs), raw_imgs

X_val_processed, X_val_raw = load_images_from_paths(filepaths, target_size=IMAGE_SIZE)


In [ ]:
input_shape = model.input_shape
print("Model expected input shape:", input_shape)
output_shape = model.output_shape
print("Model output shape:", output_shape)

In [ ]:
# Find a suitable visualization layer - use a block layer to avoid shape issues
last_conv_layer_name = None
for layer_name in ['conv5_block16_concat', 'conv5_block15_concat', 'conv5_block14_concat']:
    if any(layer.name == layer_name for layer in feature_extractor.layers):
        last_conv_layer_name = layer_name
        print(f"Using layer for visualization: {last_conv_layer_name}")
        break

# Create a model that goes up to the last conv layer
activation_model = Model(
    inputs=feature_extractor.input,
    outputs=feature_extractor.get_layer(last_conv_layer_name).output
)

# Get validation data
val_filepaths = ['split_dataset/val/' + fname for fname in val_generator.filenames[:5]]
X_val_processed, X_val_raw = load_images_from_paths(val_filepaths, target_size=(224, 224))


# Define class names
class_names = ['Mild', 'Moderate', 'No DR', 'Proliferative DR', 'Severe']

# Process validation images
for idx, (input_img, raw_img, filepath) in enumerate(zip(X_val_processed[:5], X_val_raw[:5], val_filepaths[:5])):
    print(f"Processing image {idx} - {os.path.basename(filepath)}")

    # Create input tensor
    input_tensor = tf.convert_to_tensor(np.expand_dims(input_img, axis=0), dtype=tf.float32)
    
    # Extract features for the classification model
    features = feature_extractor(input_tensor)
    print(f"Feature shape: {features.shape}")

    # Get prediction
    preds = model(features)
    class_idx = np.argmax(preds[0])
    confidence = preds[0][class_idx].numpy()
    print(f"Prediction: {class_names[class_idx]} with confidence {confidence:.4f}")

    # Compute Grad-CAM
    with tf.GradientTape() as tape:
        # Get conv output from the visualization layer
        conv_output = activation_model(input_tensor)
        tape.watch(conv_output)
        
        # Get features for classification by continuing through the rest of the model
        remaining_layers = []
        found_layer = False
        for layer in feature_extractor.layers:
            if found_layer:
                remaining_layers.append(layer)
            elif layer.name == last_conv_layer_name:
                found_layer = True
        
        # Manual forward pass through remaining layers
        x = conv_output
        for layer in remaining_layers:
            x = layer(x)
        
        # Use the classification model
        pred = model(x)
        
        # Get the score for the predicted class
        pred_class_idx = tf.argmax(pred[0])
        score = pred[0, pred_class_idx]

    # Get gradients of score with respect to conv_output
    grads = tape.gradient(score, conv_output)
    
    if grads is not None:
        # Global average pooling of gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Weight the channels by the gradient importance
        conv_output_value = conv_output[0].numpy()
        heatmap = np.zeros(conv_output_value.shape[:-1])
        
        # Weight each channel in the feature map
        for i, w in enumerate(pooled_grads):
            heatmap += w.numpy() * conv_output_value[:, :, i]
    else:
        print("Warning: Gradients are None, using feature visualization")
        # Simple feature visualization
        conv_output_value = conv_output[0].numpy()
        heatmap = np.mean(conv_output_value, axis=-1)
    
    # ReLU and normalize
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / (np.max(heatmap) + 1e-10)
    
    # Resize to image size
    heatmap = cv2.resize(heatmap, (raw_img.shape[1], raw_img.shape[0]))
    
    # Create colored heatmap
    heatmap_colored = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap_colored, cv2.COLORMAP_JET)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB) / 255.0
    
    # Superimpose on original image
    superimposed = np.clip(0.7 * raw_img + 0.3 * heatmap_colored, 0, 1)
    
    # Save and display
    class_label = class_names[class_idx]
    filename = f"{idx}_{class_label.replace(' ', '_')}.png"
    save_path = os.path.join(output_dir, filename)
    plt.imsave(save_path, superimposed)
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(raw_img)
    plt.title(f"Original - {os.path.basename(filepath)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(superimposed)
    plt.title(f"Activation: {class_label} ({confidence:.2f})")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"Saved visualization to {save_path}")

In [ ]:
# Generate visualization with ground truth labels
print("\nGenerating ground-truth based visualization grid...")

output_dir = 'split_dataset/output'
os.makedirs(output_dir, exist_ok=True)

# Use DenseNet121
feature_extractor = DenseNet121(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=(224, 224, 3)
)

feature_extractor.trainable = False

# Find a suitable visualization layer
last_conv_layer_name = None
for layer_name in ['conv5_block16_concat', 'conv5_block15_concat', 'conv5_block14_concat']:
    if any(layer.name == layer_name for layer in feature_extractor.layers):
        last_conv_layer_name = layer_name
        print(f"Using layer for visualization: {last_conv_layer_name}")
        break

# Create a model to get the activation maps
activation_model = Model(
    inputs=feature_extractor.input,
    outputs=feature_extractor.get_layer(last_conv_layer_name).output
)

# Get validation data with ground truth labels
all_val_filepaths = ['split_dataset/val/' + fname for fname in val_generator.filenames]

# Extract ground truth labels from validation generator
try:
    true_classes = val_generator.classes
    print(f"Found {len(true_classes)} labeled samples")
except:
    # If classes not available, check if folder structure indicates classes
    true_classes = []
    for path in all_val_filepaths:
        # Extract class from folder structure - adjust as needed
        parts = path.split('/')
        if len(parts) >= 3:
            class_folder = parts[-2]  # Assumes class name is the parent folder
            try:
                class_idx = class_names.index(class_folder)
            except:
                class_idx = int(class_folder) if class_folder.isdigit() else 0
            true_classes.append(class_idx)
    print(f"Extracted {len(true_classes)} classes from folder structure")

# Create dictionary to store images by ground truth class
images_by_class = {class_idx: [] for class_idx in range(len(class_names))}

# Group images by their ground truth class
for i, true_class in enumerate(true_classes):
    if len(images_by_class[true_class]) < 5:  # Only collect up to 5 images per class
        images_by_class[true_class].append(i)

# Get predictions for all validation images
all_preds = []
batch_size = 32
for i in range(0, len(all_val_filepaths), batch_size):
    batch_paths = all_val_filepaths[i:i+batch_size]
    batch_images, _ = load_images_from_paths(batch_paths, target_size=(224, 224))
    batch_tensor = tf.convert_to_tensor(batch_images, dtype=tf.float32)
    batch_features = feature_extractor(batch_tensor)
    batch_preds = model(batch_features).numpy()
    all_preds.extend(batch_preds)

# Create visualization grid
n_classes = len(class_names)
n_examples = 5
fig_width = 4 * n_examples
fig_height = 3 * n_classes * 2  # 2 rows per class (original + heatmap)

plt.figure(figsize=(fig_width, fig_height))

for class_idx in range(n_classes):
    # Get indices for this class
    class_indices = images_by_class[class_idx]

    # Check if we have images for this class
    if not class_indices:
        print(f"No images found for class {class_names[class_idx]}")
        # Create empty subplot as placeholder
        for col in range(1, n_examples+1):
            plt.subplot(n_classes * 2, n_examples, (2 * class_idx) * n_examples + col)
            plt.text(0.5, 0.5, f"No {class_names[class_idx]} examples",
                    horizontalalignment='center', verticalalignment='center')
            plt.axis('off')

            plt.subplot(n_classes * 2, n_examples, (2 * class_idx + 1) * n_examples + col)
            plt.axis('off')
        continue

    # Load images for this class
    class_paths = [all_val_filepaths[i] for i in class_indices]
    class_images, class_raws = load_images_from_paths(class_paths, target_size=(224, 224))
    class_preds = [all_preds[i] for i in class_indices]

    # Display images for this class
    for example_idx, (img_processed, img_raw, pred) in enumerate(zip(class_images, class_raws, class_preds)):
        if example_idx >= n_examples:
            break

        # Get prediction details
        pred_idx = np.argmax(pred)
        pred_prob = pred[pred_idx]

        # Create input tensor
        input_tensor = tf.convert_to_tensor(np.expand_dims(img_processed, axis=0), dtype=tf.float32)

        # Compute Grad-CAM
        with tf.GradientTape() as tape:
            # Get conv output from the visualization layer
            conv_output = activation_model(input_tensor)
            tape.watch(conv_output)
            
            # Create a model from conv output to final features
            remaining_layers = []
            found_layer = False
            for layer in feature_extractor.layers:
                if found_layer:
                    remaining_layers.append(layer)
                elif layer.name == last_conv_layer_name:
                    found_layer = True
            
            # Manual forward pass through remaining layers
            x = conv_output
            for layer in remaining_layers:
                x = layer(x)
            
            # Get prediction from classifier
            pred_tensor = model(x)
            
            # Get score for predicted class
            score = pred_tensor[0, pred_idx]
        
        # Get gradients of score with respect to conv_output
        grads = tape.gradient(score, conv_output)
        
        if grads is not None:
            # Global average pooling of gradients
            pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
            
            # Weight the channels by the gradient importance
            conv_output_value = conv_output[0].numpy()
            cam = np.zeros(conv_output_value.shape[:-1])
            
            # Weight each channel in the feature map
            for i, w in enumerate(pooled_grads):
                cam += w.numpy() * conv_output_value[:, :, i]
        else:
            # Fallback to simple feature visualization
            print(f"Warning: Gradients are None for class {class_names[class_idx]}, example {example_idx}")
            activations = activation_model(input_tensor)
            cam = np.mean(activations[0].numpy(), axis=-1)
        
        # ReLU and normalize
        cam = np.maximum(cam, 0)
        cam = cam / np.max(cam) if np.max(cam) > 0 else cam
        
        # Resize to match image size
        cam = cv2.resize(cam, (img_raw.shape[1], img_raw.shape[0]))
        
        # Create colored heatmap
        heatmap = np.uint8(255 * cam)
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB) / 255.0
        
        # Superimpose heatmap on original image
        gradcam_img = np.clip(0.7 * img_raw + 0.3 * heatmap_color, 0, 1)
        
        # Calculate grid position
        row = 2 * class_idx + 1  # Start at row 1, increment by 2 for each class
        col = example_idx + 1
        
        # Display original image
        plt.subplot(n_classes * 2, n_examples, (row - 1) * n_examples + col)
        plt.imshow(img_raw)
        
        # Show both true and predicted labels
        correct = pred_idx == class_idx
        color = 'green' if correct else 'red'
        title = f"True: {class_names[class_idx]}\nPred: {class_names[pred_idx]}"
        
        if example_idx == 0:
            title = f"Class: {class_names[class_idx]}\n{title}"
        
        plt.title(title, color=color)
        plt.axis('off')
        
        # Display Grad-CAM
        plt.subplot(n_classes * 2, n_examples, row * n_examples + col)
        plt.imshow(gradcam_img)
        plt.title(f"Conf: {pred_prob:.2f}", color=color)
        plt.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'dn121_gradcam.png'))
plt.show()
print(f"Saved dn121_gradcam visualization to {os.path.join(output_dir, 'dn121_gradcam.png')}")

Predictions for Test Set

In [ ]:
# One-hot encode labels
from tensorflow.keras.utils import to_categorical

# Assuming y_test are 1D integer class labels (0 to 4)
y_test_cat = to_categorical(y_test, num_classes=5)

In [ ]:
# Ensure output dir exists
os.makedirs(output_dir, exist_ok=True)

# Predict probabilities on test set
print("Running model inference on test set...")
start_time = time.time()
y_pred_probs_test = model.predict(X_test, verbose=1)
end_time = time.time()

# Predicted class labels
y_pred_classes_test = np.argmax(y_pred_probs_test, axis=1)

# Inference time
total_samples_test = len(X_test)
total_time_ms_test = (end_time - start_time) * 1000
avg_inference_time_test = total_time_ms_test / total_samples_test

# Compute Cohen's Kappa for test set
kappa_score_test = cohen_kappa_score(y_test, y_pred_classes_test)
print(f"\nCohen's Kappa Score (Test Set): {kappa_score_test:.4f}")

# Classification report
print("\nClassification Report (Test Set):")
report_str_test = classification_report(y_test, y_pred_classes_test, digits=4)
print(report_str_test)

with open(os.path.join(output_dir, 'classification_report_test.txt'), 'w') as f:
    f.write(report_str_test)

# Confusion matrix
cm_test = confusion_matrix(y_test, y_pred_classes_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(os.path.join(output_dir, 'confusion_matrix_test.png'))
plt.show()

# Binarize true labels for ROC curve (needed for multi-class ROC)
y_test_bin = label_binarize(y_test, classes=list(range(num_classes)))

# ROC curve & AUC for each class
fpr_test = dict()
tpr_test = dict()
roc_auc_test = dict()

for i in range(num_classes):
    fpr_test[i], tpr_test[i], _ = roc_curve(y_test_bin[:, i], y_pred_probs_test[:, i])
    roc_auc_test[i] = auc(fpr_test[i], tpr_test[i])

# Macro-average ROC
all_fpr_test = np.unique(np.concatenate([fpr_test[i] for i in range(num_classes)]))
mean_tpr_test = np.zeros_like(all_fpr_test)

for i in range(num_classes):
    mean_tpr_test += np.interp(all_fpr_test, fpr_test[i], tpr_test[i])

mean_tpr_test /= num_classes
fpr_test["macro"] = all_fpr_test
tpr_test["macro"] = mean_tpr_test
roc_auc_test["macro"] = auc(fpr_test["macro"], tpr_test["macro"])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
plt.plot(fpr_test["macro"], tpr_test["macro"],
         label=f"Macro-average ROC curve (area = {roc_auc_test['macro']:.2f})",
         color='navy', linestyle='--', linewidth=2)

colors = plt.cm.get_cmap('tab10', num_classes)
for i in range(num_classes):
    plt.plot(fpr_test[i], tpr_test[i], color=colors(i),
             lw=2, label=f"Class {i} (area = {roc_auc_test[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Multi-class Classification (Test Set)")
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'roc_curves_multiclass_test.png'))
plt.show()

# Save summary metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics_summary_test = {
    "accuracy": accuracy_score(y_test, y_pred_classes_test),
    "macro_precision": precision_score(y_test, y_pred_classes_test, average='macro'),
    "macro_recall": recall_score(y_test, y_pred_classes_test, average='macro'),
    "macro_f1": f1_score(y_test, y_pred_classes_test, average='macro'),
    "macro_roc_auc": roc_auc_test["macro"],
    "cohen_kappa": kappa_score_test,  # <-- added here
    "avg_inference_time_ms": avg_inference_time_test
}
metrics_df_test = pd.DataFrame(metrics_summary_test.items(), columns=["Metric", "Value"])
metrics_df_test.to_csv(os.path.join(output_dir, 'evaluation_metrics_test.csv'), index=False)

print("\nTest Set Evaluation Summary:")
print(metrics_df_test)